In [1]:
# preprocess data
import numpy as np
import pandas as pd


import re
import copy 
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

from IOHMM import UnSupervisedIOHMM
from IOHMM import OLS, DiscreteMNL, CrossEntropyMNL
import random


In [2]:
# construct the dataset
import torch
from torch.distributions.multivariate_normal import MultivariateNormal
from sklearn.model_selection import train_test_split
def assign_task(n):
    sequence = np.random.random(n)
    sequence[sequence>0.5] = 1
    sequence[sequence<=0.5] = 0
    return sequence
    
def covxy(x,y):
    if type(x) == list:
        x = np.array(x)
    if type(y) == list:
        y = np.array(y)
    covxy = np.mean((x - x.mean()) * (y - y.mean()))
    return covxy


N =1000
# analyze human reliance prob
query = pd.read_csv("query_.csv")
data = pd.read_csv("predictions_0110.csv")
data["taskId"] = data["taskId"].apply(lambda x:int(x))
data = data[data["taskId"]<16]
data["human_confidence"] = data["imageName"].apply(lambda x:query.query("Name==@x")["Confidence_Pred_val"].item())
data["human_confidence"] = data["human_confidence"].apply(lambda x:1 if x>0.5 else 0)
selected_data = data[data["attackType"]==0]
workers = list(set(selected_data["workerId"]))
h_a = []
h_n = []
l_a = []
l_n = []
for w in workers:
    d = data.query("workerId==@w")
    h_num = len(d.query("human_confidence==1")["reliance"])
    l_num = len(d.query("human_confidence==0")["reliance"])
    h_a.append(d.query("human_confidence==1 and attack==0")["reliance"].sum()/h_num)
    h_n.append(d.query("human_confidence==1 and attack==-1")["reliance"].sum()/h_num)
    l_a.append(d.query("human_confidence==0 and attack==0")["reliance"].sum()/l_num)
    l_n.append(d.query("human_confidence==0 and attack==-1")["reliance"].sum()/l_num)
mean = torch.tensor([np.mean(l_a),np.mean(h_a),np.mean(l_n),np.mean(h_n)])
var = torch.tensor([[covxy(l_a,l_a),covxy(l_a,h_a),covxy(l_a,l_n),covxy(l_a,h_n)],[covxy(h_a,l_a),covxy(h_a,h_a),covxy(h_a,l_n),covxy(h_a,h_n)],
                   [covxy(l_n,l_a),covxy(l_n,h_a),covxy(l_n,l_n),covxy(l_n,h_n)],[covxy(h_n,l_a),covxy(h_n,h_a),covxy(h_n,l_n),covxy(h_n,h_n)]])

dis = MultivariateNormal(mean, var)
prob = dis.sample([N])
humans = []
for i in range(prob.shape[0]):
    flag = 0
    for j in range(prob.shape[1]):
        if prob[i,j] >= 1 or prob[i,j]<=0:
            flag = 0
            break
        else:
            flag = 1
    if flag == 1:
        humans.append(prob[i,:].numpy())

task_pool = assign_task(n=20)

In [3]:
dis = MultivariateNormal(mean, var)
prob = dis.sample([N])
humans = []
for i in range(prob.shape[0]):
    flag = 0
    for j in range(prob.shape[1]):
        if prob[i,j] >= 1 or prob[i,j]<=0:
            flag = 0
            break
        else:
            flag = 1
    if flag == 1:
        humans.append(prob[i,:].numpy())
train_human, test_human, _, _ = train_test_split(humans,[0 for i in range(len(humans))], test_size=0.1, random_state=2023)
test_humans = [np.concatenate((i,i),0) for i in test_human]
# baselines [low_attack, high_attack, low_no, high_No]
W_a = [-1,-0.5,0]
W_r = [0.25,0.75,1]
C = np.linspace(0,1,11)
# no attack
def no_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            else:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            if d == 1:
                accept += 1
            else:
                reject += 1

    return w_a * accept + w_r * reject - attack * c


def all_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = n
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                d = np.random.choice([0,1],p=[1-h[1],h[1]])
            if d == 1:
                accept += 1
            else:
                reject += 1

    return w_a * accept + w_r * reject - n*len(humans) * c

def half_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
   
    for h in humans:
        tasks = assign_task(n)
        p = np.random.choice([i for i in range(n)],n//2, replace =None)
        attack_p = np.zeros(n)
        attack_p[p] = 1
        for k, t in enumerate(tasks):
            if t == 0 and attack_p[k]==0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            elif t == 1 and attack_p[k]==0:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            elif t == 0 and attack_p[k] == 1:
                attack+= 1
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                attack+=1
                d = np.random.choice([0,1],p=[1-h[1],h[1]])  
            if d == 1:
                accept += 1
            else:
                reject += 1
    return w_a * accept + w_r * reject - attack*c

def high_confidence_attack_helper(n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        tasks = assign_task(n)
        for t in tasks:
            if t == 0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            else:
                d = np.random.choice([0,1],p=[1-h[1],h[1]])
                attack += 1
            if d == 1:
                accept += 1
            else:
                reject += 1
    return w_a * accept + w_r * reject - attack * c


def eval_func(func, n, w_a, w_r, c, humans, num=50):
    u = []
    for _ in range(num):
        u.append(func(n, w_a,w_r,c,humans))
    return sum(u)/num
    
    


In [8]:
from statsmodels.tools import add_constant

def trust_transition(model, inputs, state_src, state_dst):
#     inputs = add_constant(inputs,has_constant='add')
    prob = np.exp(model.model_transition[state_src].predict_log_proba(inputs)).reshape(model.num_states)
    return prob[state_dst]

def reliance_prob(model, inputs, state):
#     inputs = add_constant(inputs,has_constant='add')

    return np.exp(model.model_emissions[state][0].predict_log_proba(inputs)).reshape(2)


def reward(model, state, confidence,attack, w_a, w_r, c):
    inputs = np.array([[confidence, attack]])

    r = 0
    for i in range(model.num_states):

        r += trust_transition(model, inputs, state, i)* (reliance_prob(model, inputs,i)[0]*w_r +
                                                         reliance_prob(model, inputs,i)[1]*w_a - attack*c)
    return r
        
def reward_belief(model, belief, confidence,attack, w_a, w_r, c):
    r = 0
    for i in range(belief.shape[0]):
        r += belief[i] * reward(model, i, confidence, attack, w_a, w_r, c)
    return r

def EU_max(model, belief, confidence, attack, l, w_a, w_r, c,max_length=2):
    if l < 1:
        print("error: l should be greated than 1")
    if l == 1:
        eu_max = reward_belief(model, belief, confidence, attack, w_a, w_r, c)
        return eu_max
    else:
        r = reward_belief(model, belief, confidence, attack, w_a, w_r, c)
        eu_max = r 
        for d in [0,1]:
            sum_ = 0
            updated_b = update_belief(model, belief, confidence, attack,reliance=d)
            for k in range(belief.shape[0]):
                b_k = belief[k]
                inputs = np.array([[ confidence, attack]])
                for i in range(model.num_states):
                    
                    sum_ += b_k*trust_transition(model, inputs, k, i)*(reliance_prob(model,inputs,i)[d]*max(0.5*(EU_max(model, updated_b,confidence=0,attack=0,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)
                                                                                                              +EU_max(model, updated_b,confidence=1,attack=0,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)),
                                                                                    0.5*(EU_max(model, updated_b,confidence=0,attack=1,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c)+EU_max(model, updated_b,confidence=1,attack=1,l=min(l-1,max_length),w_a=w_a, w_r=w_r, c=c))))
            eu_max += sum_
        return eu_max

def update_belief(model,belief, confidence, attack, reliance, constant = 1):
    updated_b = np.zeros_like(belief)
    inputs = np.array([[confidence, attack]])
    for j in range(belief.shape[0]):
        for i in range(belief.shape[0]):
            updated_b[j] += constant*belief[i]*trust_transition(model, inputs, state_src=i,state_dst=j)*reliance_prob(model, inputs, j)[reliance]
    return updated_b
    
def covxy(x,y):
    if type(x) == list:
        x = np.array(x)
    if type(y) == list:
        y = np.array(y)
    covxy = np.mean((x - x.mean()) * (y - y.mean()))
    return covxy

def hmm_attack_helper(model, n, w_a, w_r, c, humans):
    accept = 0
    reject = 0
    attack = 0
    for h in humans:
        b = np.array([1/model.num_states for i in range(model.num_states)])

        tasks = assign_task(n)
        for k, t in enumerate(tasks):
            
            action = np.argmax([EU_max(model, b, t, 0, n-k, w_a, w_r, c),EU_max(model, b, t, 1, n-k, w_a, w_r, c)])
            
            if t == 0 and action==0:
                d = np.random.choice([0,1],p=[1-h[2],h[2]])
            elif t == 1 and action==0:
                d = np.random.choice([0,1],p=[1-h[3],h[3]])
            elif t == 0 and action == 1:
                attack+= 1
                d = np.random.choice([0,1],p=[1-h[0],h[0]])
            else:
                attack+=1
                d = np.random.choice([0,1],p=[1-h[1],h[1]]) 
            b = update_belief(model, b, t,action, d)
            if d == 1:
                accept += 1
            else:
                reject += 1
            print(accept, reject, attack)
    return w_a * accept + w_r * reject - attack*c
    
    
    
    

In [9]:
#eval hmm-based algorithm

#1. contruct human behavior data
hmm_data = selected_data
behavior_data = []
hmm_data["reliance"] = hmm_data["reliance"].apply(lambda x: 1 if x == True else -1)
hmm_data["attack"] = hmm_data["attack"].apply(lambda x:1 if x == 0 else x)
hmm_data["human_confidence"] = hmm_data["human_confidence"].apply(lambda x:-1 if x == 0 else x)
workers = list(set(selected_data["workerId"]))
for w in workers:
    d = hmm_data.query("workerId==@w").sort_values(by=["taskId"])
    behavior_data.append(d[["human_confidence","attack", "reliance"]])
initial_columns = ["human_confidence", "attack"]
transition_columns = ["human_confidence", "attack"]
decision_columns = ["human_confidence", "attack"]

model = UnSupervisedIOHMM(num_states=2, max_EM_iter=200, EM_tol=1e-6)

model.set_models(model_emissions = [DiscreteMNL(solver='lbfgs')], 
                model_transition=CrossEntropyMNL(solver='lbfgs'),
                model_initial=CrossEntropyMNL(solver='lbfgs'))

model.set_inputs(covariates_initial = initial_columns, covariates_transition = initial_columns, covariates_emissions = [initial_columns])

model.set_outputs([['reliance']])

model.set_data(behavior_data)
model.train()
# hmm_attack_helper(model, 20,0,0.5,0.15, test_humans)

In [12]:
C = [i *0.01 for i in range(10,20)]
for c in C:
    print("attack cost:---->{}".format(c))
    print("high confidence:{}".format(eval_func(high_confidence_attack_helper,20,0,0.5,c, test_humans)))
    print("half attack:{}".format(eval_func(half_attack_helper,20,0,0.5,c, test_humans)))
    print("all attack:{}".format(eval_func(all_attack_helper,20,0,0.5,c, test_humans)))
    print("no attack:{}".format(eval_func(no_attack_helper, 20,0,0.5,c, test_humans)))

attack cost:---->0.1
high confidence:233.34999999999997
half attack:251.58
all attack:292.18
no attack:210.84
attack cost:---->0.11
high confidence:227.87999999999997
half attack:247.7700000000002
all attack:280.7500000000002
no attack:212.66
attack cost:---->0.12
high confidence:225.83479999999992
half attack:242.75000000000014
all attack:272.43999999999977
no attack:210.09
attack cost:---->0.13
high confidence:219.0494000000001
half attack:237.30999999999986
all attack:262.80000000000024
no attack:213.71
attack cost:---->0.14
high confidence:213.49839999999992
half attack:232.40999999999977
all attack:251.93999999999983
no attack:213.22
attack cost:---->0.15
high confidence:209.10500000000008
half attack:226.57
all attack:242.66
no attack:212.83
attack cost:---->0.16
high confidence:204.248
half attack:221.15000000000018
all attack:233.7600000000001
no attack:213.03
attack cost:---->0.17
high confidence:198.60959999999994
half attack:218.96000000000006
all attack:225.0499999999998
no

In [ ]:
C = [i *0.01 for i in range(10,20)]
for c in C:
    print("cost:{}, utility:{}".format(c, hmm_attack_helper(model, 20,0,0.5,c, test_humans)))

0 1 1
0 2 2
0 3 3
0 4 4
0 5 5
0 6 6
0 7 7
0 8 8
1 8 9
1 9 10
1 10 11
1 11 12
2 11 13
2 12 14
2 13 15
2 14 16
2 15 17
2 16 18
2 17 19
3 17 20
3 18 21
3 19 22
4 19 23
4 20 24
4 21 25
4 22 26
4 23 27
5 23 28
5 24 29
5 25 30
5 26 31
5 27 32
5 28 33
5 29 34
5 30 35
6 30 36
6 31 37
6 32 38
6 33 39
6 34 40
6 35 41
7 35 42
8 35 42
9 35 42
9 36 42
9 37 42
9 38 43
9 39 44
10 39 45
10 40 46
11 40 47
11 41 48
11 42 49
11 43 50
11 44 51
12 44 52
12 45 53
12 46 54
12 47 55
12 48 56
12 49 57
12 50 58
12 51 59
12 52 60
12 53 61
12 54 62
12 55 63
13 55 64
13 56 65
14 56 66
15 56 67
15 57 67
16 57 67
16 58 68
16 59 69
16 60 70
17 60 71
17 61 72
18 61 73
18 62 74
19 62 75
19 63 75
19 64 75
19 65 76
20 65 77
21 65 78
21 66 78
22 66 79
23 66 79
23 67 79
24 67 79
24 68 80
24 69 81
24 70 82
24 71 83
24 72 84
24 73 85
25 73 86
25 74 87
25 75 88
25 76 89
25 77 90
26 77 91
26 78 92
27 78 93
27 79 94
27 80 95
27 81 96
27 82 97
28 82 98
28 83 99
28 84 100
29 84 101
29 85 102
30 85 103
30 86 104
30 87 105
30 88 10

184 557 636
184 558 637
184 559 638
184 560 639
184 561 640
184 562 641
184 563 642
184 564 643
184 565 644
184 566 645
184 567 646
184 568 647
184 569 648
184 570 649
184 571 650
184 572 651
184 573 652
184 574 653
184 575 654
184 576 655
184 577 656
184 578 657
184 579 658
184 580 659
184 581 660
184 582 661
184 583 662
184 584 663
184 585 664
184 586 665
184 587 666
184 588 667
184 589 668
184 590 669
184 591 670
184 592 671
184 593 672
184 594 673
184 595 674
185 595 675
186 595 676
186 596 676
187 596 676
188 596 676
189 596 676
189 597 676
190 597 677
190 598 677
191 598 677
192 598 677
193 598 677
194 598 677
194 599 677
194 600 677
194 601 678
194 602 679
194 603 680
194 604 681
195 604 682
195 605 683
195 606 684
195 607 685
195 608 686
196 608 687
197 608 688
197 609 689
197 610 690
197 611 691
197 612 692
197 613 693
197 614 694
198 614 695
199 614 696
200 614 696
200 615 696
201 615 696
201 616 696
202 616 697
202 617 697
202 618 698
202 619 699
202 620 700
202 621 701
202 